## B3情報科学演習：時系列データ分析（前編）

---

### はじめに

実際に研究などのプロジェクトに取り組む際には, "長期的な見通し" を持つことが重要である  
例えば, 以下のような点をある程度の粒度で抑えておくことで, 矛盾を防ぐことができる  
- 概要・目的  
- 実施条件  
- 実施手法  
- 評価方法  

※特に共同研究や規模の大きなプロジェクトなど, 複数人で進めていく場合に効果を実感する

例に倣って, 前提条件を設定する

#### 前提条件

- 概要・目的   
　― 気温, 風速, 風向などの様々な要素(説明変数)から, 日射量(目的変数)を推定する学習モデルを作成する
- 実施条件  
　― 過去, 現在の説明変数から, 現在の目的変数を推定する  
- 実施手法  
　― 学習器には, RandomForestとLSTMを試す  
　― 他の学習器は別途各自調べて, 実装してみるのもあり  
- モデルの評価方法  
　― 決定係数（R^2）と 二乗平均平方根誤差（RMSE）  

それでは, これから実際に"データ処理"のコードを書いていく  
流れとしては,  
「ライブラリのインポート」  
→「データ読み込み」  
→「データクレンジング」  
→「データ前処理」  
→「特徴量エンジニアリング(FE)」  
→「データ探索分析(EDA)」  
→「データ保存」  
という流れで進める

---

### ライブラリのインポート

使用するライブラリを最初にインポートする  
最初にインポートするのは後から特定のセルを実行する際に，それより前のセルをすべて実行する必要をなくすためである

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pytz
from pytz import timezone
from tqdm import tqdm
from tsfresh.feature_extraction import extract_features

---

### データ読み込み

今回はExcelファイルで渡されたため、pandasのread_excel()で読み込む

In [2]:
org_data = pd.read_excel('./data/trainval_data.xlsx')
org_data.head()

,UNIX時間,日付,現地時間,日射量[w/㎡],華氏[ºF],気圧[Hg],湿度[%],風向[度],風速[mph],日の出時刻,日の入り時刻
0,1472724008,9/1/2016,00:00:08,2.58,51,30.43,103,77.27,11.25,06:07:00,18:38:00
1,1472724310,9/1/2016,00:05:10,2.83,51,30.43,103,153.44,9.00,06:07:00,18:38:00
2,1472725206,9/1/2016,00:20:06,2.16,51,30.43,103,142.04,7.87,06:07:00,18:38:00
3,1472725505,9/1/2016,00:25:05,2.21,51,30.43,103,144.12,18.00,06:07:00,18:38:00
4,1472725809,9/1/2016,00:30:09,2.25,51,30.43,103,67.42,11.25,06:07:00,18:38:00


情報系の人はcsvやtsv形式を好む人が多いが、一般的にはアプリケーションで扱いやすいExcelファイルで渡されることが多い  
本当に酷い場合はpdf形式で渡されることがあると就活時のインターン先の方から聞いたことがあるが，仕方がないこと  
極力，対処法を自分で見つけて，本当に無理なら他のデータ形式で頂けないかデータ提供者に相談する

---

### データクレンジング

データ提供型の共同研究では少し使いづらいデータ形式で渡されることがあるので，最初に自分が使いやすい形式に変換を行なう  
Excelソフトを使って手動で編集しても良いが，データ量が多いと処理に時間がかかったり，間違いがあったときに再度始めから行うのが面倒（マクロを使うのもあり）  
データ処理もスクリプト化して追加データにも対応可能にした方が良い

まず, 日本語でも使用できるが面倒な時があるため英語に変換する  
長すぎず分かりやすい英数字表記にする  
DataFrame.columnsに直接代入しても良いが、分かりやすくrename()を使用する

In [ ]:
jp_to_en = {
    'UNIX時間':'UNIX_Time',
    '日付':'Date',
    '現地時間':'Time',
    '日射量[w/㎡]':'Radiation',
    '華氏[ºF]':'Temprature',
    '気圧[Hg]':'Pressure',
    '湿度[%]':'Humidity',
    '風向[度]':'Wind_Direction',
    '風速[mph]':'Wind_Speed',
    '日の出時刻':'SunRise_Time',
    '日の入り時刻':'SunSet_Time'
}
trainval_data = org_data.copy()
trainval_data.rename(columns=jp_to_en, inplace=True)
trainval_data.head()

---

### データ前処理

センサデータを扱う際は基本的に連続値を使うことが多いが，まれに離散値が混じる場合も十分あり得る  
連続値は特に考えずそのまま使ってよいが，離散値は何かしら前処理が必要になることが多い  

In [ ]:
#データ型の確認
trainval_data.dtypes

今回のデータでは, "Date"と"Time", "SunRise_Time", "SunSet_Time"の4つの説明変数がオブジェクト型となっており, 何等かの処理を施すことで学習に使用できるように変換する必要がある  

まずは, "Date"と"Time"だが, これらの説明変数は"UNIX_Time"という説明変数と役割が被っているため, 正直必要がない  
"UNIX_Time"はint型であり, 都合がよいため, 今回はこの説明変数を残すことにする  
また, "UNIX_Time"をpandasに用意されているdatetime64型という時系列データを扱う上で処理で役に立つ型へ変換し, インデックスに指定する  

In [ ]:
#UNIX_Timeをdatetime型に変換し, indexに指定
hawaii = timezone('Pacific/Honolulu')
trainval_data.index = pd.to_datetime(trainval_data['UNIX_Time'], unit='s')
trainval_data.drop(['UNIX_Time','Date', 'Time'], axis=1, inplace=True)
trainval_data.index = trainval_data.index.tz_localize(pytz.utc).tz_convert(hawaii)

次に, "SunRise_Time"と"SunSet_Time"だが, "UNIX_Time"と同様にDatetime64型に変換しておくことで, 時系列データ処理が行いやすい形に変換しておく

In [ ]:
#object型のカラムの変換
trainval_data['SunRise_Time'] = pd.to_datetime(trainval_data['SunRise_Time'], format='%H:%M:%S')
trainval_data['SunSet_Time'] = pd.to_datetime(trainval_data['SunSet_Time'], format='%H:%M:%S')
trainval_data.head()

次に, データに存在する欠損値を確認する

In [ ]:
#欠損値の確認
trainval_data.isnull().sum()

今回のデータではどの変数にも欠損値が存在していないことが確認できたため, 欠損値を処理する手間が省けた  

欠損値の扱いには色々ある．以下は例
- 時間的に前後のデータから補間(欠損規模に依存)
- 他の変数を参考に似ているデータから計算(例: 1年前の同日の値を使用)
- そもそも欠損値の時点データを使わない(無難)

ただし，欠損値の処理はテストデータには行わない  
補間した値を使って『学習』する分のは自由だが，補間した値を使って『予測』するのは自分で作ったデータを予測していることになるため，あまり適切でない  

---

### 特徴量エンジニアリング(FE:Feature Engineering)

機械学習の『性能向上を目的とした』前処理を行う（ズルはダメ）  
ここでの処理は使用する学習器やそもそもの目的によって大きく変わる  
無限に考えられるのでここでは適当にやってみる  
だけど最近多く用いられる深層学習はそもそも特徴量エンジニアリングから学習までを行うものであるため不要とも考えられる

#### 不要カラムの削除

説明変数は多いほど良いというものでもない(多重共線性の誘発)し, 少なければ良いというものでもない(モデルの表現力低下)  
そのため, 最適な説明変数を模索していく必要がある  
まずは, 学習精度向上にほとんど意味の無いと思う(むしろ無い方が精度が高くなることもある)説明変数を削除してみる  
学習時の特徴量重要度等を出力してみて, その結果から削除する流れの方が自然だが, 精度ばかり気にして, 残った説明変数が合理性に欠けてしまい, 妥当な学習モデルになっていない可能性もあることに注意  

例) アイスクリームの売り上げを予測するモデルの説明変数に「季節」や「気温」等を削除し, 「水難事故の件数」にした方が精度が上がったとして, それは妥当だと言えるか？(疑似相関)


今回は, とりあえずデモとして"Wind_Direction"という説明変数を削除してみる。(本当は重要な説明変数かも？)  

In [ ]:
#不要カラムの削除
trainval_data.drop('Wind_Direction', axis=1, inplace=True)
trainval_data.head()

---

#### 記述統計量の追加

最大，最小，平均，分散など記述統計量を特徴量として追加する  
集約範囲は過去1日，1週間，1か月などいろいろ  
経験的にはあまり効果的でないことが多いが，とりあえず色んな変数の記述統計量を入れて学習して，結果から取捨選択する手もある  
理論上は, 外れ値等に強くなりロバストな学習が実現したり, 目的変数にフィットする説明変数に化ける可能性もある  
また，記述統計量ではないが積算値や階差値も利用可能

今回は, "Temprature"という説明変数の1次階差(一つ前時点との差)を説明変数として加えてみる  
前時点を考慮した予測になってくれればいいなくらいの効果を淡く期待している

In [ ]:
# Tempratureの1次階差を追加
trainval_data['Temprature_diff'] = trainval_data['Temprature'] - trainval_data['Temprature'].shift(1)
trainval_data.dropna(how='any',axis=0, inplace=True)
trainval_data.head()

---

#### 時間情報を追加

時間情報が大きく影響する場合有効  
ただし，時間という数字はあくまで人間が作り出したものと考えるとあまり適切ではない気もする  

今回は, Datetime64型の"SunRise_Time"と"SunSet_Time"という説明変数があったため, 組み合わせて一つの説明変数にしてみようと思う  
発想としては, 日の出から日の入りまでの時間があるなら, 「日が出ている時間」という説明変数を生み出すことができるのではないかという経緯である  

ということで, "SunSet_Time"と"SunRise_Time"の差を"Day_Length"という名前の説明変数として追加し, 多重共線性を回避するため, "SunRise_Time"と"SunSet_Time"を削除する

In [ ]:
trainval_data['Day_Length'] = trainval_data['SunSet_Time'].dt.hour*60*60 + trainval_data['SunSet_Time'].dt.minute*60 + trainval_data['SunSet_Time'].dt.second \
                           - trainval_data['SunRise_Time'].dt.hour*60*60 - trainval_data['SunRise_Time'].dt.minute*60 - trainval_data['SunRise_Time'].dt.second
trainval_data.drop(['SunRise_Time', 'SunSet_Time'], axis=1, inplace=True)
trainval_data.head()

---

#### tsfresh事件

去年の情報科学演習(現在のB4がB3だった時)の時系列パートで起こった事件の一つを以下で紹介します。読み飛ばしてもらっても構いません  

事件の始まりは, 「"tsfresh"という時系列特徴量を大量に自動生成してくれるライブラリもあるので使ってみるといいかも」という先輩の意見からだった  
そんな便利なライブラリがあるなら使うしかない！と思った当時B3のR.O君はtsfreshをインストールして実際に使用した結果を進捗報告で紹介した(以下, 当時の会話内容)  

R.O君「使ってみたが, 結論:tsfreshは使えないことが分かった」↓当時の発表スライド↓  

<img src='./image/image1.jpg' width=600>

峰野先生「でもさぁ～！, 作った人もさぁ～！何かに使えるから作ったと思うんだよね～。すぐに決めつけちゃうのは研究者として良くないと思うな！」  

R.O君「はぁ、、」  

R.O君はその後しばらく, 峰野先生から, 使えないことが分かったと決めつけるキャラとしてイジられることとなった  
これ以降R.O君の口からtsfreshという言葉を聞くことは無くなったのである、、  

tsfreshが本当は使える可能性もまだ何%か残っているかもしれないので, 時間があれば試してみてほしい

適当に特徴量エンジニアリング（FE）をやってみた  
FEは前の章でやったデータ前処理と異なり，『機械学習にとって都合の良い特徴量を生成すること』が目的になるため，意味が分からないけど優秀な特徴量を生成してみてもいい  
ただし，ここでの処理も一応データ提供者側に妥当かどうか確認を行う方がいいと思う  
また，単純に精度が高いモデルを作るのが目的なら良いものの，その特徴量自体が目的に近い存在ならばよく考えてFEする必要があるのは当然

---

### データ探索分析(EDA:Explanatory Data Analysis)

どのようなデータを使うかを確認する  
最初はとりあえずいくつか可視化してみてデータの特徴，傾向をつかむ  
機械学習を使用するということはまだ置いといて，ドメイン知識や分析結果を用いてデータの前処理行う

#### 折れ線グラフ

折れ線グラフをプロットする  
注意点としては離散値はそのままでは数値型でなくプロットできない

In [ ]:
cmap = plt.get_cmap('tab10')
plt.figure(figsize=[13,10])
for i, c in enumerate(trainval_data.columns):
    plt.subplot(6,2,i+1)
    plt.plot(trainval_data.index, trainval_data[c], c=cmap(i%10), linewidth=1)
    plt.title(c)
plt.tight_layout()

#### ヒストグラム

箱ひげ図よりもより細かく分布を把握できる  
ビンの数によって凹凸具合が割と簡単に変わりやすいので注意

In [ ]:
cmap = plt.get_cmap('tab10')
plt.figure(figsize=[13,10])
for i, c in enumerate(trainval_data.columns):
    plt.subplot(6,2,i+1)
    sns.histplot(data=trainval_data, x=c, bins=30, color=cmap(i%10), kde=True)
    plt.title(c)
plt.tight_layout()

#### ヒートマップ

DataFrame.corr()で各カラム間の相関係数を計算してくれる（数値型以外は無視）  
seabornのheatmapに入れるだけで分かりやすく可視化

In [ ]:
corr = trainval_data.corr()
plt.figure(figsize=[10,8])
sns.heatmap(corr, cmap=sns.color_palette('coolwarm', 10), annot=True, fmt='.2f', vmin = -1, vmax = 1)

---

### データ確認

In [ ]:
trainval_data

---

### データ保存

ここまでの様々な工程で加工されたデータを新たにprocessed_dataという名称でcsvとして保存する

In [ ]:
trainval_data.to_csv('./data/processed_data.csv')

残りは学習モデルを使って予測を行っていく  
モデルの学習はmodel.ipynbで行う